In [1]:
#foundamental package
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# multivariate lstm example
#model related packages
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import sklearn.model_selection as model_selection
#Metrics packages
from sklearn import metrics, svm, datasets 
from sklearn.metrics import confusion_matrix,precision_score, recall_score, f1_score, accuracy_score
#othrt packages
from timeit import default_timer as timer
import warnings
warnings.filterwarnings("ignore")

In [2]:
from  glassnode  import  GlassnodeClient
import pandas as pd

api_key = "24vuPNOyd3ciclDMDBqFCoQFykc"
client = GlassnodeClient(api_key)
# since = 1514822399 # 1 1 2019
# until = 1646841599 # 1 3 2022
since = 1546272000 # 1 1 2019
until = 1646064000 # 1 3 2022
resolution = "24h"
params = {"a": "BTC", "s": since, "u": until, "i": resolution}
indlist = [['market','price_usd_close'],
          ['addresses','min_1_count'],
          ['addresses','min_10_count'],
          ['addresses','min_100_count'],
          ['addresses','min_1k_count'],
          ['addresses','min_10k_count'],
          ['addresses','active_count'],
          ['addresses','new_non_zero_count'],
           ['addresses','receiving_count'],
           #cant use
           ['distribution','supply_contracts'],
           ['distribution','balance_exchanges'],
           ['distribution','balance_exchanges_relative'],
           ['distribution','exchange_net_position_change'],
           #cant use
           ['supply','rcap_hodl_waves'],
           ['supply','active_3m_6m'],
           ['supply','active_6m_12m'],
           ['supply','active_1y_2y'],
           ['supply','active_2y_3y'],
           ['supply','active_3y_5y'],
           ['supply','profit_relative'],
           ['supply','profit_sum'],
           ['supply','loss_sum'],
           #do not have this data
           ['supply','sth_lth_realized_value_ratio'],
           #cant use
           ['transactions','count'],
           ['transactions','transfers_volume_exchanges_net'],
           ['market','mvrv'],
           ['market','mvrv_z_score'],
           ['market','marketcap_realized_usd'],
           ['indicators','pi_cycle_top'],
           ['indicators','net_unrealized_profit_loss'],
           ['indicators','seller_exhaustion_constant'],
           ['indicators','realized_profits_to_value_ratio'],
           ['indicators','realized_profit'],
           ['indicators','realized_loss'],
           ['indicators','ssr'],
           ['indicators','ssr_oscillator'],
           ['lightning','network_capacity_sum'],
           #cant use this data
           ['protocols','uniswap_volume_sum']
           #['',''],          
          ]


def data_from_api(indlist):
    data=[]
    for i in range(len(indlist)):
        try:
            label = indlist[i][1]
            db = client.get(indlist[i][0],indlist[i][1],params)
            #convert from list of dictionaries to data frame
            df = pd.DataFrame.from_dict(db,orient='columns')
            #setup columns name
            df.columns=['time',label]
            #fixed index
            df.set_index('time', inplace = True)
            data.append(df)
            data_df = pd.concat(data,axis = 1)
        except:
            print('data can be used',label)
    data_df['time'] = data_df.index
    data_df['time_real'] = pd.to_datetime(data_df['time'],unit = 's')
    #set time as the first columns
    data_df.set_index('time_real')
    # data_df= data_df[['time_real']+[col for col in data_df if col != 'time' and 'time_real' ]]
 #   df = df[ ['mean'] + [ col for col in df.columns if col != 'mean' ] ]
    return data_df


In [3]:
df_raw = data_from_api(indlist)
# df_raw.head(10)

data can be used supply_contracts
data can be used sth_lth_realized_value_ratio
data can be used uniswap_volume_sum


In [4]:
# 选出 'rcap_hodl_waves','pi_cycle_top','ssr' 中的数据
df = df_raw.iloc[:,:].drop(columns = ['rcap_hodl_waves','pi_cycle_top','ssr'])

#features 处理
df['1_10_address'] =   df['min_1_count'] - df['min_10_count']
df['10-100-address'] = df['min_10_count'] - df['min_100_count']
df['100-1k-address'] = df['min_100_count'] - df['min_1k_count']
df['1k-10k-address'] = df['min_1k_count'] - df['min_10k_count']
df['10k+address'] =    df['min_10k_count']

df["per_1_d"] = df['price_usd_close'].pct_change(1)
df["per_3_d"] = df['price_usd_close'].pct_change(3)
df["per_7_d"] = df['price_usd_close'].pct_change(7)
df["per_15_d"] = df['price_usd_close'].pct_change(15)
df["per_30_d"] = df['price_usd_close'].pct_change(30)
df['price_back_1_d'] = df['price_usd_close'].shift(periods=1)
df['price_back_3_d'] = df['price_usd_close'].shift(periods=3)
df['price_back_7_d'] = df['price_usd_close'].shift(periods=7)
df['price_back_15_d'] = df['price_usd_close'].shift(periods=15)
df['price_back_30_d'] = df['price_usd_close'].shift(periods=30)
df['SMA_3'] = df['price_usd_close'].rolling(window=3).mean()
df['SMA_7'] = df['price_usd_close'].rolling(window=7).mean()
df['up_down'] = df["per_1_d"]
#condition = [(abs(df["per_1_d"])<0),(df["per_1_d"]>=0)]
condition = [(abs(df["per_1_d"])<0.005),(df["per_1_d"]<=-0.005),(df["per_1_d"]>=0.005)]
values = [-1,0,1]
import numpy as np
df['up_do_1d'] = np.select(condition,values)
#drop columns and store this dataset                                         
df1 = df.drop(['min_1_count','min_10_count','min_100_count','min_1k_count','min_10k_count'],axis=1)
#for SVM 1 day price prediction
df1 = df1.drop(['per_1_d','per_3_d','per_7_d','per_15_d','per_30_d','price_back_1_d','price_back_3_d','price_back_7_d','price_back_15_d','price_back_30_d'],axis=1)
#df1.iloc[30:,:]
#df1.to_csv(r'G:\MScProject\data\raw_add_col.csv')



In [25]:
df2 = df1.iloc[30:,]
df2_up_do_1d = df2['up_do_1d']
df2_price_usd_close = df2['price_usd_close']
df2_time_real = df2['time_real']
df2_up_down = df2['up_down']
#set index
df2 = df2.set_index('time_real')
# df2 = df2.iloc[30:,]


df2 = df2.drop(['up_do_1d','up_down','time'],axis = 1)
df2
df2_time_real

time
1548806400   2019-01-30
1548892800   2019-01-31
1548979200   2019-02-01
1549065600   2019-02-02
1549152000   2019-02-03
                ...    
1645574400   2022-02-23
1645660800   2022-02-24
1645747200   2022-02-25
1645833600   2022-02-26
1645920000   2022-02-27
Name: time_real, Length: 1125, dtype: datetime64[ns]

In [6]:
# l = len(norm_df)
# time_train,time_test = df2_time_real[:int(0.8*l)],df2_time_real[int(0.8*l):]
# # test_index = pd.DataFrame(test_index)
# # print(df2_time_real)
# # print(time_train)
# # time_test  = pd.DataFrame(time_test).reset_index().drop('time',axis=1)
# time_test=pd.DataFrame(time_test).reset_index().drop('time',axis=1)
# time_test

In [7]:
#papare dataset 
# normaliztion
price_re = df2['price_usd_close'].max()-df2['price_usd_close'].min()
price_min = df2['price_usd_close'].min()
norm_df=(df2-df2.min())/(df2.max()-df2.min())
norm_df.head(10)
# norm_df.shape

# change the first 'price_usd_close' to the last column
norm_df= norm_df[[col for col in norm_df if col != 'price_usd_close']+['price_usd_close']]
norm_df.head(10)

,active_count,new_non_zero_count,receiving_count,balance_exchanges,balance_exchanges_relative,exchange_net_position_change,active_3m_6m,active_6m_12m,active_1y_2y,active_2y_3y,...,ssr_oscillator,network_capacity_sum,1_10_address,10-100-address,100-1k-address,1k-10k-address,10k+address,SMA_3,SMA_7,price_usd_close
time_real,,,,,,,,,,,,,,,,,,,,,
2019-01-30,0.131128,0.222141,0.157291,0.375201,0.615787,0.427906,0.144897,0.426643,0.782853,0.022057,...,0.297781,0.000000,0.000000,0.427924,0.943516,0.051020,0.461538,0.000690,0.001365,0.001320
2019-01-31,0.166722,0.262763,0.200676,0.394855,0.633628,0.470181,0.155905,0.421583,0.787319,0.021748,...,0.299730,0.004967,0.005693,0.353313,0.948290,0.053936,0.461538,0.000610,0.001024,0.000882
2019-02-01,0.195515,0.250170,0.226422,0.398196,0.636267,0.490433,0.158051,0.416038,0.792775,0.022311,...,0.305066,0.006639,0.009118,0.353313,0.919650,0.049563,0.538462,0.000830,0.000763,0.001372
2019-02-02,0.067233,0.154717,0.080558,0.389836,0.628053,0.506036,0.159534,0.411490,0.798721,0.022533,...,0.309808,0.008225,0.013052,0.351356,0.917263,0.051020,0.538462,0.001050,0.000590,0.001966
2019-02-03,0.029465,0.110129,0.031463,0.392499,0.630121,0.499628,0.163028,0.408964,0.799654,0.027215,...,0.308203,0.008986,0.016706,0.354024,0.908512,0.052478,0.538462,0.001072,0.000302,0.000949
2019-02-04,0.138938,0.208596,0.145944,0.391577,0.628824,0.480811,0.156495,0.398739,0.811898,0.028488,...,0.310809,0.010435,0.021252,0.360071,0.916468,0.048105,0.538462,0.000904,0.000266,0.000875
2019-02-05,0.178307,0.300413,0.227702,0.389970,0.626889,0.465743,0.149086,0.398853,0.817861,0.028747,...,0.314800,0.011025,0.026045,0.354558,0.921241,0.042274,0.538462,0.000555,0.000297,0.000937
2019-02-06,0.221130,0.261593,0.191704,0.394019,0.630169,0.473039,0.147541,0.392752,0.822734,0.028812,...,0.312243,0.014171,0.022246,0.349489,0.929196,0.046647,0.564103,0.000297,0.000130,0.000191
2019-02-07,0.174289,0.247411,0.178194,0.409692,0.644234,0.506325,0.152661,0.390226,0.824783,0.028804,...,0.313921,0.016145,0.028806,0.360427,0.922037,0.039359,0.564103,0.000000,0.000000,0.000000


In [35]:
norm_df.to_csv(r'G:\MScProject\data\norm_df.csv')

In [8]:
#---------------------------------------------------------------model training---------------------------------------------------------------#
#---------------------------------------------------------------model training---------------------------------------------------------------#
#---------------------------------------------------------------model training---------------------------------------------------------------#

In [9]:
def lstm_model(X,X_train, y_train,n_steps,fst_lyer,act_fun,fst_dense,epochs):
    #steps can be 1,3,5,7
    #fst_lyer can be 10,25,50,100,200
    #act_fun :'tanh','relu'
    #fst_dense : 1,3,5,
    #epochs:20,50,100,200
    
        n_features = X.shape[2]
        # define model
        model = Sequential()
        model.add(LSTM(fst_lyer, activation=act_fun, input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        # fit model
        model.fit(X_train, y_train, epochs=epochs, verbose=0)
        
        return model
        

In [16]:
#---------------create squences data--------------------#
def split_sequences(sequences, n_steps):
    #normalization dataset from data propocessing
    # n_step:1,3,5
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences.iloc[i:end_ix, :-1], sequences.iloc[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [26]:
#----------------------------deal with  data after get the normalization data
def deal_with_result(X,df2_time_real,y_test,y_pred):
    #df2_time_real from data processing
    #y_test :from data split
    #y_pred : from data split step in main function
    l_x = len(X)
    split_point = int(0.8*l_x)
        
    # l=len(df2_time_real)
    time_train,time_test = df2_time_real[:split_point],df2_time_real[split_point:]
    # test_index = pd.DataFrame(test_index)
    time_test  = pd.DataFrame(time_test).reset_index().drop('time',axis=1)
    # print(time_test)

    y_test = pd.DataFrame(y_test)
    y_pred = pd.DataFrame(y_pred)
    result = pd.concat([time_test,y_test,y_pred],axis = 1)
    result.columns = ['time_test','y_test','y_pred']
    # result['time_test'].astype(str)
    # ,columns = ['test_index','y_test','y_pred']
    # test_index = pd.Index('time_test')
    # result = result.set_index(test_index)
    # result = result.iloc[:,1:]
    re_result = result[:]
    re_result['y_pred'] = re_result['y_pred']* price_re + price_min
    re_result['y_test'] = re_result['y_test']* price_re + price_min
    re_result['gap_act'] = re_result['y_test']
    re_result['gap_pred'] = re_result['y_test']
    re_result['gap_act_err'] = re_result['y_test']
    re_result['gap_pred_err'] = re_result['y_test']
    
    for i in range(len(re_result)-1):
        re_result['gap_act'][i+1] = re_result['y_test'][i+1]-re_result['y_test'][i]
        re_result['gap_pred'][i+1] = re_result['y_pred'][i+1]-re_result['y_test'][i]
        re_result['gap_act_err'][i+1] = re_result['gap_act'][i+1]/re_result['y_test'][i]
        re_result['gap_pred_err'][i+1] = re_result['gap_pred'][i+1]/re_result['y_test'][i]
        
    re_result = re_result[1:]
    # print(re_result)
    
    
    
        #create 3 classes actual error
    condition = [(abs(re_result["gap_act_err"])<=0.005),(re_result["gap_act_err"]< -0.005),(re_result["gap_act_err"]> 0.005)]
    values = [-1,0,1]
    import numpy as np
    re_result['gap_act_err_class_3'] = np.select(condition,values)

    #create 3 classes prediction error
    condition = [(abs(re_result["gap_pred_err"])<=-0.005),(re_result["gap_pred_err"]< -0.005),(re_result["gap_pred_err"]> 0.005)]
    values = [-1,0,1]
    import numpy as np
    re_result['gap_pred_err_class_3'] = np.select(condition,values)

    #create 3 classes actual error
    condition = [(re_result["gap_act_err"])<=0,(re_result["gap_act_err"]>0)]
    values = [-1,1]
    import numpy as np
    re_result['gap_act_err_class_2'] = np.select(condition,values)

    #create 3 classes actual error
    condition = [(re_result["gap_pred_err"]<=0),(re_result["gap_pred_err"]>0)]
    values = [-1,1]
    re_result['gap_pred_err_class_2'] = np.select(condition,values)
    print(re_result.head(10))
    print(re_result.tail(10))
    
    return re_result


In [27]:
#---------------------------- metrics calculate-------------------------#
def LSTM_metric(re_result,num_class):
    # inpput the result from model prediction, and the number of class you want to calculate
    #re_result :previou step
    # num_calss: 3 defalut
    met_lst = []
    
    if num_class==2:
        y_true=re_result['gap_act_err_class_2']
        y_pred=re_result['gap_pred_err_class_2']
    else:
        y_true=re_result['gap_act_err_class_3']
        y_pred=re_result['gap_pred_err_class_3']
    
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred)
    acc = accuracy_score(y_true, y_pred)
    f1_weight = f1_score(y_true, y_pred,average = 'weighted')
    recall = recall_score(y_true, y_pred,average = 'weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    met_lst.append(acc)
    met_lst.append(f1_weight)
    met_lst.append(recall)
    met_lst.append(precision)
    # print(met_lst)
    
    #return a list[accuracy,f1,recall,precision]
    return met_lst


In [28]:
#time series corss validation
def lstm_para_tune(dataset,n_steps,folds,fst_lyer,act_fun,fst_dense,epochs,df2_time_real,num_class):
    #norm_df : from norm_df
    # X, y = split_sequences(dataset, n_steps)
    
    para_lst = []
    l_d = len(dataset)
    block_size = int(l_d/folds)
    for i in range(1,folds+1):
#-----------dataset for ith fold---------------------#
        i_fold_data = dataset[0:i*block_size]
        df2_time_real_new = df2_time_real[0:i*block_size] 
        X, y = split_sequences(i_fold_data, n_steps)
#-----------split to train and test set---------------#         
        l_x = len(X)
        split_point = int(0.8*l_x)
        X_train, X_test = X[:split_point],X[split_point:]
        y_train, y_test = y[:split_point],y[split_point:]
        time_train,time_test = df2_time_real_new[:split_point],df2_time_real_new[split_point:]
        # print('data split for % folds training'.format(i))
        
#---------tuning model paras--------------------------------#
        model = lstm_model(X,X_train, y_train,n_steps,fst_lyer,act_fun,fst_dense,epochs)
        y_pred = model.predict(X_test, verbose=0)
        # print('model training for % folds training'.format(i))
        
#--------result------------------------------------------#
        # l_test = len()

        re_result = deal_with_result(X,df2_time_real_new,y_test,y_pred)
        # print('model training for % folds training'.format(i))
        
#------------metrics calculate---------------#
        
        met_lst = LSTM_metric(re_result,num_class)
    
        para_lst.append([folds,i,n_steps,fst_lyer,act_fun,epochs,met_lst[0],met_lst[1],met_lst[2],met_lst[3]])
        
                        
    
    para_lst_df = pd.DataFrame (para_lst, columns = ['fold', 'i_round','n_steps','fst_lyer','act_fun','epochs','acc','f1','recall','precision'])
    
    return para_lst_df
                        
                       

In [34]:
lstm_para_tune(norm_df,1,5,50,'relu',1,200,df2_time_real,3)

    time_test        y_test        y_pred     gap_act    gap_pred  \
1  2019-07-30   9604.742004   9730.826172   83.250091  209.334259   
2  2019-07-31  10085.627749  10125.380859  480.885745  520.638855   
3  2019-08-01  10399.669423  10425.546875  314.041674  339.919126   
4  2019-08-02  10518.174677  10261.646484  118.505254 -138.022939   
5  2019-08-03  10821.726510  10749.958008  303.551833  231.783331   
6  2019-08-04  10970.184216  10698.812500  148.457705 -122.914010   
7  2019-08-05  11805.653725  11792.498047  835.469510  822.313831   
8  2019-08-06  11478.169346  11356.894531 -327.484379 -448.759194   
9  2019-08-07  11941.968341  11629.476562  463.798995  151.307216   
10 2019-08-08  11966.407405  11667.719727   24.439064 -274.248614   

    gap_act_err  gap_pred_err  gap_act_err_class_3  gap_pred_err_class_3  \
1      0.008743      0.021985                    1                     1   
2      0.050068      0.054206                    1                     1   
3      0.031

,fold,i_round,n_steps,fst_lyer,act_fun,epochs,acc,f1,recall,precision
0,5,1,1,50,relu,200,0.613636,0.562238,0.613636,0.544609
1,5,2,1,50,relu,200,0.696629,0.623894,0.696629,0.570425
2,5,3,1,50,relu,200,0.298507,0.180745,0.298507,0.494306
3,5,4,1,50,relu,200,0.469274,0.317520,0.469274,0.644013
4,5,5,1,50,relu,200,0.517857,0.435004,0.517857,0.603608


In [33]:
lstm_para_tune(norm_df,2,10,50,'relu',1,200,df2_time_real,3)

    time_test       y_test       y_pred     gap_act     gap_pred  gap_act_err  \
1  2019-04-29  5350.032501  4943.562500  101.828230  -304.641771     0.019402   
2  2019-04-30  5401.029636  4891.322266   50.997135  -458.710236     0.009532   
3  2019-05-01  5505.283672  4904.852051  104.254036  -496.177585     0.019303   
4  2019-05-02  5768.289407  4833.605469  263.005735  -671.678203     0.047773   
5  2019-05-03  5829.939030  4823.150879   61.649623  -945.138528     0.010688   
6  2019-05-04  5790.998759  4996.956543  -38.940272  -832.982487    -0.006679   
7  2019-05-05  5750.260655  5062.051758  -40.738103  -728.947001    -0.007035   
8  2019-05-06  5843.495066  5048.938477   93.234411  -701.322179     0.016214   
9  2019-05-07  5983.760613  4825.437988  140.265546 -1018.057078     0.024004   
10 2019-05-08  6174.132348  4721.487305  190.371735 -1262.273308     0.031815   

    gap_pred_err  gap_act_err_class_3  gap_pred_err_class_3  \
1      -0.058047                    1        

,fold,i_round,n_steps,fst_lyer,act_fun,epochs,acc,f1,recall,precision
0,10,1,2,50,relu,200,0.347826,0.179523,0.347826,0.120983
1,10,2,2,50,relu,200,0.422222,0.276470,0.422222,0.389664
2,10,3,2,50,relu,200,0.492537,0.438877,0.492537,0.419242
3,10,4,2,50,relu,200,0.700000,0.622850,0.700000,0.577984
4,10,5,2,50,relu,200,0.348214,0.227266,0.348214,0.525033
5,10,6,2,50,relu,200,0.340741,0.253682,0.340741,0.542291
6,10,7,2,50,relu,200,0.375796,0.240555,0.375796,0.676333
7,10,8,2,50,relu,200,0.463687,0.299987,0.463687,0.646789
8,10,9,2,50,relu,200,0.628713,0.587742,0.628713,0.585717
9,10,10,2,50,relu,200,0.696429,0.628346,0.696429,0.573242


In [203]:
lstm_para_tune(norm_df,1,10,100,'relu',1,200,df2_time_real,3)

    time_test       y_test       y_pred     gap_act     gap_pred  gap_act_err  \
13 2019-05-12  6972.371523  6036.947754 -239.921656 -1175.345425    -0.033266   
14 2019-05-13  7807.134381  6382.169922  834.762858  -590.201601     0.119724   
15 2019-05-14  7994.416156  6667.373047  187.281775 -1139.761334     0.023989   
16 2019-05-15  8205.167495  6996.585938  210.751339  -997.830219     0.026362   
17 2019-05-16  7883.392905  6702.521484 -321.774590 -1502.646010    -0.039216   
18 2019-05-17  7332.333628  6661.423828 -551.059277 -1221.969077    -0.069901   
19 2019-05-18  7272.870365  6520.936523  -59.463264  -811.397105    -0.008110   
20 2019-05-19  8197.081550  6790.785156  924.211185  -482.085208     0.127077   
21 2019-05-20  7977.281728  6749.485352 -219.799822 -1447.596198    -0.026814   
22 2019-05-21  7964.087168  6775.866211  -13.194560 -1201.415517    -0.001654   

    gap_pred_err  gap_act_err_class_3  gap_pred_err_class_3  \
13     -0.162964                    0        

,fold,i_round,n_steps,fst_lyer,act_fun,epochs,acc,f1,recall,precision
0,10,1,1,100,relu,200,0.318182,0.153605,0.318182,0.101240
1,10,2,1,100,relu,200,0.454545,0.344099,0.454545,0.505114
2,10,3,1,100,relu,200,0.582090,0.519099,0.582090,0.505804
3,10,4,1,100,relu,200,0.730337,0.653189,0.730337,0.608751
4,10,5,1,100,relu,200,0.396396,0.307207,0.396396,0.495495
5,10,6,1,100,relu,200,0.335821,0.250457,0.335821,0.542727
6,10,7,1,100,relu,200,0.358974,0.211999,0.358974,0.675381
7,10,8,1,100,relu,200,0.474860,0.323474,0.474860,0.649187
8,10,9,1,100,relu,200,0.776119,0.733084,0.776119,0.695229


In [204]:
lstm_para_tune(norm_df,1,10,75,'relu',1,200,df2_time_real,3)

    time_test       y_test       y_pred     gap_act    gap_pred  gap_act_err  \
13 2019-05-12  6972.371523  6848.698730 -239.921656 -363.594449    -0.033266   
14 2019-05-13  7807.134381  7438.621582  834.762858  466.250059     0.119724   
15 2019-05-14  7994.416156  7545.228516  187.281775 -261.905866     0.023989   
16 2019-05-15  8205.167495  7705.462891  210.751339 -288.953266     0.026362   
17 2019-05-16  7883.392905  7592.834961 -321.774590 -612.332534    -0.039216   
18 2019-05-17  7332.333628  7288.061523 -551.059277 -595.331382    -0.069901   
19 2019-05-18  7272.870365  7124.989258  -59.463264 -207.344370    -0.008110   
20 2019-05-19  8197.081550  7768.144531  924.211185  495.274167     0.127077   
21 2019-05-20  7977.281728  7672.530273 -219.799822 -524.551276    -0.026814   
22 2019-05-21  7964.087168  7657.737305  -13.194560 -319.544424    -0.001654   

    gap_pred_err  gap_act_err_class_3  gap_pred_err_class_3  \
13     -0.050413                    0                   

,fold,i_round,n_steps,fst_lyer,act_fun,epochs,acc,f1,recall,precision
0,10,1,1,75,relu,200,0.590909,0.575494,0.590909,0.775568
1,10,2,1,75,relu,200,0.613636,0.549156,0.613636,0.561527
2,10,3,1,75,relu,200,0.626866,0.550984,0.626866,0.494823
3,10,4,1,75,relu,200,0.719101,0.639054,0.719101,0.579242
4,10,5,1,75,relu,200,0.432432,0.354085,0.432432,0.485928
5,10,6,1,75,relu,200,0.320896,0.227343,0.320896,0.583940
6,10,7,1,75,relu,200,0.371795,0.237152,0.371795,0.676940
7,10,8,1,75,relu,200,0.480447,0.334841,0.480447,0.650407
8,10,9,1,75,relu,200,0.601990,0.567989,0.601990,0.545552


In [128]:
lstm_para_tune(norm_df,1,15,100,'relu',1,200,df2_time_real,3)

,fold,i_round,n_steps,fst_lyer,act_fun,epochs,acc,f1,recall,precision
0,15,1,1,100,relu,200,0.992243,0.988379,0.992243,0.984545
1,15,2,1,100,relu,200,0.982823,0.974309,0.982823,0.965941
2,15,3,1,100,relu,200,0.969048,0.953815,0.969048,0.939053
3,15,4,1,100,relu,200,0.955123,0.933199,0.955123,0.912260
4,15,5,1,100,relu,200,0.934605,0.903013,0.934605,0.873486
5,15,6,1,100,relu,200,0.928571,0.894180,0.928571,0.862245
6,15,7,1,100,relu,200,0.920467,0.882347,0.920467,0.847259
7,15,8,1,100,relu,200,0.898148,0.849955,0.898148,0.806670
8,15,9,1,100,relu,200,0.859873,0.795088,0.859873,0.739381
9,15,10,1,100,relu,200,0.804255,0.717001,0.804255,0.646827


In [ ]:
paras=[[],
            [],
            [],
            [],
            []
           ]

In [ ]:
for i in len(paras[0])